# Stock Screening (In Trade) v0

In this notebook we try to achieve
- Given a list of Ticker, compare Volumn from 2 day ago til today
- Send Telegram notice

Timeline as follow:

|| T0 | T1 | T2 | T3 |
| --- | --- | --- | --- | --- |
| Expected trend | x | Volume Rise | Volume Drop | Price Rise |
| Action | x | Create Potential Stocks List | **Enter Market near day end** | |

Import Library

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

Import Token

In [ ]:
from secrets import IEX_SANDBOX_API_TOKEN as IEX_CLOUD_API_TOKEN
baseUrl = f'https://sandbox.iexapis.com/stable'

# from secrets import IEX_CLOUD_API_TOKEN
# baseUrl = f'https://cloud.iexapis.com/stable'

IEX Cloud API test call

In [ ]:
symbol = 'AAPL'
symbolString = 'AAPL,TSLA,CANB'
symbolString = 'ABCL,ACHR,ACMR,ACTG'

tradingDates = f'/ref-data/us/dates/trade/last/?token={IEX_CLOUD_API_TOKEN}'

apiUrl = baseUrl + tradingDates
# data = requests.get(apiUrl)
# print(data.status_code)
# print(data.text)
data = requests.get(apiUrl).json()
data

## Preparation

Fetch ticker of "today" watch list (which is generated in last trading day).

In which will also define "T0 date" of this screening.

In [ ]:
apiUrl = baseUrl + f'/ref-data/us/dates/trade/last/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(apiUrl).json()
data

In [ ]:
lastTradingDate = data[0]['date'].replace("-", "")
lastTradingDate

In [ ]:
potentialStocks = pd.read_csv(f'../export/{lastTradingDate}/yau4muk6man4_strategy_US_{lastTradingDate}_volume_rise.csv')
potentialStocks

In [ ]:
t0Date = potentialStocks.loc[0, 'T0 Date'].replace("-", "")
t0Date

Define chunks to prepare for batch API call

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
symbolGroups = list(chunks(potentialStocks['Ticker'], 100))
symbolStrings = []
for i in range(0, len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
#     print(symbol_strings[i])

symbolStrings

## Data Fetching

Create main DataFrame, including volume of today, previous trading date and the day before.

Data imported from IEX Cloud API.

In [ ]:
finalDataFrameColumns = [
    'Ticker',
    'Price',
    '30 day Average Volume',
    'T0 Date',
    'T0 Volume',
    'T1 Volume',
    'T1 Volume Ratio',
    'T2 Volume',
    'T2 Volume Ratio',
]

finalDataFrame = pd.DataFrame(columns = finalDataFrameColumns)

for symbolString in symbolStrings:
    batchApiUrlCall = baseUrl + f'/stock/market/batch?symbols={symbolString}&types=quote,chart&exactDate={t0Date}&chartCloseOnly=true&chartByDay=true&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiUrlCall).json()
#     print(data)
#     print(data.status_code)
    for symbol in symbolString.split(','):
        try:
            latestPrice = data[symbol]['quote']['latestPrice']
        except KeyError:
            latestPrice = np.NaN
        try:
            avgTotalVolume = data[symbol]['quote']['avgTotalVolume']
        except KeyError:
            avgTotalVolume = np.NaN
        try:
            latestVolume = data[symbol]['quote']['latestVolume']
        except KeyError:
            latestVolume = np.NaN

        try:
            t1Volume = data[symbol]['quote']['previousVolume']
        except (KeyError, TypeError):
            t1Volume = np.NaN
            
        try:
            t0Volume = data[symbol]['chart'][0]['volume']
        except IndexError:
            t0Volume = np.NaN

        finalDataFrame = finalDataFrame.append(
            pd.Series(
                [
                    symbol,
                    latestPrice,
                    avgTotalVolume,
                    data[symbol]['chart'][0]['date'],
                    t0Volume,
                    t1Volume,
                    'N/A',
                    latestVolume,
                    'N/A',
                ],
                index = finalDataFrameColumns
            ),
            ignore_index = True
        )

finalDataFrame

## Data Cleaning

Clean out data with None from API

In [ ]:
finalDataFrame[finalDataFrame.isnull().any(axis = 1)]
# finalDataFrame[finalDataFrame['Price'].isnull()]

In [ ]:
finalDataFrame = finalDataFrame.dropna()
finalDataFrame

## Calculation

Calculate Volume change Ratio

In [ ]:
for row in finalDataFrame.index:
    finalDataFrame.loc[row, 'T1 Volume Ratio'] = finalDataFrame.loc[row, 'T1 Volume'] / finalDataFrame.loc[row, 'T0 Volume']
    finalDataFrame.loc[row, 'T2 Volume Ratio'] = finalDataFrame.loc[row, 'T2 Volume'] / finalDataFrame.loc[row, 'T1 Volume']
    
finalDataFrame

## Screening

Screening criteria are as follow:

- T2 Volume is below 25% of T1

In [ ]:
t2traget = float(0.25)

finalDataFrame = finalDataFrame[
    (finalDataFrame['T2 Volume Ratio'] < t2traget)
]
finalDataFrame.sort_values(
    'Ticker',
    ascending = True,
    inplace = True
)
finalDataFrame.reset_index(inplace = True, drop = True)

finalDataFrame

## Data Export

In [ ]:
from datetime import date
import os

today = date.today().strftime("%Y%m%d")
folderName = f'../export/{today}'
fileName = f'{folderName}/yau4muk6man4_strategy_US_{today}_volume_drop'
folderName = f'../export/{today}/'
csvFileName = f'{fileName}.csv'
xlsxFileName = f'{fileName}.xlsx'
sheetName = f'{today} Volume Drop'

In [ ]:
try:
    os.mkdir(folderName)
    print("Directory " , folderName ,  " Created ") 
except FileExistsError:
    print("Directory " , folderName ,  " already exists")

Export to xlsx

In [ ]:
writer = pd.ExcelWriter(xlsxFileName, engine = 'xlsxwriter')
finalDataFrame.to_excel(writer, sheet_name = sheetName, index = False)

In [ ]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'#,###',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

floatTemplate = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [ ]:
columnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['30 day Average Volume', integerTemplate],
    'D': ['T0 Date', stringTemplate],
    'E': ['T0 Volume', integerTemplate],
    'F': ['T1 Volume', integerTemplate],
    'G': ['T1 Volume Ratio', percentTemplate],
    'H': ['T2 Volume', integerTemplate],
    'I': ['T2 Volume Ratio', percentTemplate],
}

for column in columnFormats.keys():
    writer.sheets[sheetName].set_column(
        f'{column}:{column}',
        max(len(columnFormats[column][0]), 10),
        columnFormats[column][1]
    )
    writer.sheets[sheetName].write(
        f'{column}1',
        columnFormats[column][0],
        columnFormats[column][1]
    )

In [ ]:
writer.save()